#### Irrigation system Model 


In [4]:
# imports 
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import datetime
import pathlib
from datetime import datetime
from dataclasses import dataclass
import math


In [ ]:
def reynolds_pipe(density, velocity, diameter, viscosity):
    """Calculate the Reynolds number for a given pipe flow.
        density:density of the fluid in [kg/m3]
        velocity:the mean velocity of the fluid in [m/s]
        viscosity:dynamic viscosity of the fluid in [Pa*s]
        diameter:inside diameter of the pipe in [m]
    """
    return density * velocity * diameter / viscosity


def friction_factor(Re, e, D):
    """ Calculate the friction for the Darcy-weisback formula
        Re: reynolds number 
        e: roughness factor 
        D: inner diameter of the pip
    """  
    
    if Re < 2000:   #Laminar flow
        f = 64/Re 
    if Re > 4000:   #Turbulent flow 
        f = 0.25/ (math.log10((e)/(3.7*D)+5.74/(Re**0.9)))**2
    else:  #Dunlop(1991) cubic interpolation formula for moody diagram
        R = Re/2000
        AB = 0.00328895476345399058690
        AA = −1.5634601348517065795
        Y2 = (e/(3.7*D)) + AB
        Y3 = -2 * math.log(Y2)
        FA = (Y3)**(-2)
        FB = FA * (2 - (AA * AB)/(Y2* Y3))
        X1 = 7* FA - FB
        X2 = 0.128 -17*FA +2.5*FB
        X3 = -0.128 + 13*FA -2*FB
        X4 = 0.032 - 3*FA +0.5*FB
        f = X1 + R*(X2 + R*(X3+R*X4)) 
    return f 

def pipe_head_losses(Q,D,L, e, rho = 1000, viscosity=10**(-3), g = 9.81):
    '''calculate the head losses of a pipe section
        Q: flow rate through pipe section 
        D: diameter of pipe section 
        e: rougness of pipesection 
        L:length of pipe section
        rho: density of fluid[kg/m3] --> default water 
        viscosity: viscosity of fluid [Pa.s] --> default water 
        g :gravitational acceleration [m/s2]
    '''
    v = Q / (math.pi * (D/2)**2)
    Re = reynolds_pipe(rho,v,D,viscosity)
    f = friction_factor(Re,e,D)
    return (f * L * v^2)/ (2*D*g) #Darcy-weisbach 

In [ ]:
def network_head_losses(L_main, L_submain, L_lateral, n_lateral, n_emitters_per_plant, n_plants_per_lateral):
    ''' Function that calculates the head loss of a drip irrigation network given the following inputs:
        L_main: the length of the main pipe (m)
        L_submain: the length of the submain pipe (m)  
        L_lateral: the length of each of the lateral pipes (m)
        n_lateral: the number of lateral pipes in the network --> should be even number 
        n_emitters_per_plant: number of drip emitters per plant
        n_plants_per_lateral: number of plants per lateral, assumed to be evenly spaced 
        
    '''
    #water constants
    rho = 1000 #density of water[kg/m3]
    viscosity = 10**(-3) #viscosity of water [Pa.s]
    g = 9.81 #gravitational acceleration [m/s2]

    #network components 
    n_tees = n_lateral-1 #number of t-joints in the network
    n_elbows = 2 #1 elbow connecting each of the end laterals 
    n_valve = 1 # at begining of network 
    n_emiiters_per_lateral = n_emitters_per_plant * n_plants_per_lateral
    D_main = #Inner diameter of main [mm]
    D_submain = #Inner diameter of submain [mm]
    D_lateral = #Inner diameter of lateral [mm]
    Q_rated = #the rated flow rate of the drip irrigatiate [m3/h] 

    #Minor loss coeficients --> sourced from ...  
    k_tee_straight = 
    k_tee_bend = 
    k_elbow = 
    k_valve = 
    k_plug = #at the end of each lateral 

    H_minor = k  

    # mannings rougness coeficient  --> sourced from...
    e_main = 
    e_submain = 
    e_lateral = 

    Q_total = (Q_rated * n_emitters_per_plant * n_plants_per_lateral * n_lateral) / 3600 #total flow rate [m3/s]

    #Losses in main 
    H_main = pipe_head_losses(Q_total,D_main,L_main, e_main)

    #Losses in submain 
    submain_parts = n_lateral/2 #each side, so actually 2 times this value
    L_submain_parts = L_submain/(submain_parts*2 +1)
    H_submain = 0 #intiate as 0 
    for i in range(submain_parts):
        Q_part = (Q_total/2)/(submain_parts) * (submain_parts-i) #flow rate through each section 
        H_submain += 2*pipe_head_losses(Q_part,D_submain,L_submain_parts, e_submain) 

    #Losses in laterals 
    lateral_parts = n_emiiters_per_lateral #number of equal length segments between each emitter
    L_lateral_parts = L_lateral/ lateral_parts
    H_lateral = 0 #intiate as 0
    for i in range(lateral_parts):
        Q_part_lateral = (Q_total/n_lateral) - (Q_rated/3600) * i #flow rate through each section 
        H_lateral += n_lateral*pipe_head_losses(Q_part_lateral,D_lateral,L_lateral_parts, e_lateral)  


    

    
